<a href="https://colab.research.google.com/github/ozgurugurr/Colab-Multi-Class-Text-Classification/blob/master/MachineLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Description X CategoryCode ML çalışması yaklaşık 4 sa sürüp 20 GB RAM kullanılmıştır.

# --# Naive Bayes Classifier for Multinomial #
# acc : 0.55

# -- # Linear Support Vector Machine #
# acc : 0.64

# --# Logistic Regression # 
# acc : 0.74

# -- # Doc2vec and Logistic Regression # 
# acc :  some fix needed

In [0]:
!sudo apt-get update
%cd drive/My\ Drive/tf-ıdf-fasttext

In [0]:
import os
import spacy
import pickle
from spacy.lang.en import English
from spacy import displacy
import numpy as np
import tqdm
import pandas as pd
from gensim.models import FastText
from gensim.models.phrases import Phrases, Phraser
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation, strip_non_alphanum
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import logging
from numpy import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import nltk
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
import gensim
from gensim.models.doc2vec import TaggedDocument
import re
%matplotlib inline

In [0]:
# Veriyi drive'a taşı
# Mouth Drive

In [0]:
pd.options.display.max_colwidth = 800
path = "/content/drive/My Drive/tf-ıdf-fasttext/AllMRLineItems_Mapping_Matched.csv"
combined = pd.read_csv(path, skip_blank_lines= False, sep=',', low_memory=False)
combined = pd.DataFrame(data = combined)
combined = combined[['MRLineItemDescription','CategoryCode']]
combined.head()

In [0]:
print("CategoryCode sayısı\n")
combined.CategoryCode.nunique()

In [0]:
combined = combined[pd.notnull(combined['CategoryCode'])]
print(combined.head(10))
print("Toplam kelime sayısı")
print(combined['MRLineItemDescription'].apply(lambda x: len(x.split(' '))).sum())

In [0]:
# homojen bir dağılım gözlemlenmedi

In [0]:
my_tags = ['ATRX', 'DD00', 'AFGB', 'DB00', 'AMSC', 'AFCL', 'ATWP', 'AFWC', 'ATJS', 'AFTC', 'AYTC', 'AFCP', 'AY00', 'ANNM', 'AEFK', 'AFTS', 'MZW0', 'U000', 'ALLU', 'AFFT', 'ADDW', 'CE00', 'EWE1', 'ER00', 'EEA0', 'EF00', 'AYSS', 'EA00', 'PY22', 'EP00', 'PX01', 'MDHM', 'PF03', 'NND0', 'USME', 'AEFS', 'ADGS', 'ADDX', 'MLXR', 'ATFS', 'ADHD', 'AYVL', 'ANEJ', 'AFPP', 'AERA', 'AEDL', 'ADDZ', 'AFCA', 'MHLF', 'AYSF', 'ALST', 'MGED', 'ESL1', 'ETP3', 'ESL2', 'EMT2', 'ESL0', 'ES00', 'ECK0', 'EYW0', 'ENA0', 'EU00', 'MWBS', 'AYVR', 'MAHU', 'MPVE', 'MAHT', 'CY03', 'MPGF', 'PF02', 'MEC0', 'MPGC', 'PY26', 'MPGR', 'MAH0', 'ME00', 'SV00', 'JN01', 'MAHC', 'PY32', 'EVV1', 'JQ12', 'AYTA', 'AFFR', 'AFFL', 'AYML', 'ADRC', 'AYMP', 'AICM', 'AYMS', 'AESP', 'AYBD', 'AFFA', 'PY21', 'AIWS', 'AIWH', 'AFFW', 'EG00', 'ERG0', 'ELF1', 'EGL0', 'ELF2', 'EFC0', 'EWG1', 'ACE0', 'PX12', 'PB03', 'NNP0', 'MWKW', 'PV01', 'PV00', 'MPGL', 'JF05', 'ATPW', 'MFFW', 'ENF0', 'MAHV', 'ATIB', 'DD01', 'ATR0', 'ETV0', 'MPGH', 'MAHM', 'ATCT', 'AYPA', 'CY10', 'ADCA', 'DY00', 'ALLG', 'AICK', 'EFV5', 'EMT1', 'ALLS', 'PB00', 'PY05', 'MPGS', 'AAPG', 'AFMP', 'AFGG', 'MSXF', 'ADDS', 'EUY0', 'EMM0', 'MBA0', 'MEVH', 'ATRL', 'MDNE', 'NLLG', 'AFPS', 'EFW0', 'AFFC', 'ELF0', 'AEPC', 'ATWH', 'MERS', 'ATRT', 'AYPW', 'ATWC', 'MBS0', 'DY10', 'DG03', 'EB00', 'MEHU', 'DG00', 'AFMI', 'ADSM', 'ADDF', 'AMUM', 'MWDH', 'EH01', 'PY17', 'PY00', 'AAFS', 'EFV2', 'DY11', 'ATWF', 'AFPE', 'AFGP', 'AD00', 'AFFZ', 'CY17', 'MDND', 'PY01', 'DG01', 'SS01', 'CP00', 'NWC0', 'EL00', 'ERT0', 'ETP1', 'EWU0', 'FB02', 'MWR0', 'MEHR', 'MXR0', 'JL00', 'MPVS', 'MPC0', 'EBB3', 'EFY0', 'EJB0', 'EQ00', 'ET00', 'EZL0', 'EZ00', 'MERC', 'MLGP', 'MPVC', 'MLXF', 'MT00', 'PX05', 'PB05', 'PV37', 'PX09', 'PY02', 'PY10', 'MJKG', 'AILB', 'AFPC', 'MML0', 'D000', 'AI00', 'AA00', 'ATFR', 'M000', 'J000', 'ATPR', 'MT0F', 'AFPI', 'AFCM', 'AFFN', 'ECK1', 'EEC0', 'AC00', 'PY24', 'MDD0', 'MEP0', 'PV03', 'MT0W', 'JP05', 'MPCG', 'PV07', 'PY30', 'MD00', 'AE00', 'DE00', 'EW00', 'EJ00', 'ENS0', 'EWL0', 'ERF0', 'EFN7', 'EFE1', 'EFE2', 'JQ05', 'EFP0', 'ED00', 'PP00', 'MEEW', 'ANNH', 'EMP6', 'ENG0', 'ETP0', 'EWJ0', 'JA03', 'JA34', 'JD01', 'MBPR', 'MC00', 'MCRA', 'MJKH', 'MPGB', 'MPPM', 'MPVW', 'MWDC', 'PS02', 'PB04', 'NW00', 'ACCH', 'ATWB', 'AIAH', 'AFWF', 'ATRJ', 'ADEA', 'EE00', 'ECM4', 'ERC1', 'AYWP', 'AIAR', 'ATWM', 'ATDB', 'SS20', 'AESC', 'EFD0', 'MFD0', 'EWU1', 'MP00', 'MSXH', 'MUSG', 'ESM0', 'EWC0', 'JV07', 'MDNS', 'EBB1', 'P000', 'MECM', 'MPGW', 'PY14', 'MMT0', 'MUTC', 'PX07', 'ERC5', 'AFFS', 'EMT0', 'EZS0', 'MBSD', 'AMUG', 'AKSN', 'AYPC', 'ADGG', 'ESI1', 'EMC2', 'FA00', 'MFDR', 'AMCR', 'PP02', 'PP01', 'DG02', 'C000', 'EAA1', 'ECM5', 'ALS0', 'AELH', 'PV09', 'JP00', 'AFWV', 'ERC3', 'PX00', 'JT00', 'PP03', 'MJLL', 'ERC8', 'JD00', 'JA02', 'MEHX', 'ATRE', 'ADWA', 'ENN1', 'EGA0', 'EAD1', 'EFV0', 'ADDE', 'MEEP', 'MBFH', 'ERX0', 'JP01', 'NN00', 'MHC0', 'EY00', 'DA01', 'MTCT', 'SS25', 'AMSR', 'A000', 'MAA0']
plt.figure(figsize=(30,8))
plt.xlabel("Category Codes", fontsize=24)
plt.xticks(rotation=30)
plt.ylabel("Count Of MR Line Item",fontsize=24)
combined.CategoryCode.value_counts().plot(kind='bar');

In [0]:
def print_plot(index):
    example = combined[combined.index == index][['MRLineItemDescription', 'CategoryCode']].values[0]
    if len(example) > 0:
        print(example[0])
        print('CategoryCode:', example[1])
print("Veri kontrolü")
print_plot(10)

In [0]:
nltk.download('stopwords')
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = BeautifulSoup(text, "lxml").text 
    text = text.lower() 
    text = REPLACE_BY_SPACE_RE.sub(' ', text) #
    text = BAD_SYMBOLS_RE.sub('', text) 
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) 
    return text
    
combined['MRLineItemDescription'] = combined['MRLineItemDescription'].apply(clean_text)
print_plot(10)

In [0]:
print("Ön işlemeden geçmiş veride bulunan kelime sayısı\n")
combined['MRLineItemDescription'].apply(lambda x: len(x.split(' '))).sum()

In [0]:
X = combined.MRLineItemDescription
y = combined.CategoryCode
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)
my_tags = ['ATRX', 'DD00', 'AFGB', 'DB00', 'AMSC', 'AFCL', 'ATWP', 'AFWC', 'ATJS', 'AFTC', 'AYTC', 'AFCP', 'AY00', 'ANNM', 'AEFK', 'AFTS', 'MZW0', 'U000', 'ALLU', 'AFFT', 'ADDW', 'CE00', 'EWE1', 'ER00', 'EEA0', 'EF00', 'AYSS', 'EA00', 'PY22', 'EP00', 'PX01', 'MDHM', 'PF03', 'NND0', 'USME', 'AEFS', 'ADGS', 'ADDX', 'MLXR', 'ATFS', 'ADHD', 'AYVL', 'ANEJ', 'AFPP', 'AERA', 'AEDL', 'ADDZ', 'AFCA', 'MHLF', 'AYSF', 'ALST', 'MGED', 'ESL1', 'ETP3', 'ESL2', 'EMT2', 'ESL0', 'ES00', 'ECK0', 'EYW0', 'ENA0', 'EU00', 'MWBS', 'AYVR', 'MAHU', 'MPVE', 'MAHT', 'CY03', 'MPGF', 'PF02', 'MEC0', 'MPGC', 'PY26', 'MPGR', 'MAH0', 'ME00', 'SV00', 'JN01', 'MAHC', 'PY32', 'EVV1', 'JQ12', 'AYTA', 'AFFR', 'AFFL', 'AYML', 'ADRC', 'AYMP', 'AICM', 'AYMS', 'AESP', 'AYBD', 'AFFA', 'PY21', 'AIWS', 'AIWH', 'AFFW', 'EG00', 'ERG0', 'ELF1', 'EGL0', 'ELF2', 'EFC0', 'EWG1', 'ACE0', 'PX12', 'PB03', 'NNP0', 'MWKW', 'PV01', 'PV00', 'MPGL', 'JF05', 'ATPW', 'MFFW', 'ENF0', 'MAHV', 'ATIB', 'DD01', 'ATR0', 'ETV0', 'MPGH', 'MAHM', 'ATCT', 'AYPA', 'CY10', 'ADCA', 'DY00', 'ALLG', 'AICK', 'EFV5', 'EMT1', 'ALLS', 'PB00', 'PY05', 'MPGS', 'AAPG', 'AFMP', 'AFGG', 'MSXF', 'ADDS', 'EUY0', 'EMM0', 'MBA0', 'MEVH', 'ATRL', 'MDNE', 'NLLG', 'AFPS', 'EFW0', 'AFFC', 'ELF0', 'AEPC', 'ATWH', 'MERS', 'ATRT', 'AYPW', 'ATWC', 'MBS0', 'DY10', 'DG03', 'EB00', 'MEHU', 'DG00', 'AFMI', 'ADSM', 'ADDF', 'AMUM', 'MWDH', 'EH01', 'PY17', 'PY00', 'AAFS', 'EFV2', 'DY11', 'ATWF', 'AFPE', 'AFGP', 'AD00', 'AFFZ', 'CY17', 'MDND', 'PY01', 'DG01', 'SS01', 'CP00', 'NWC0', 'EL00', 'ERT0', 'ETP1', 'EWU0', 'FB02', 'MWR0', 'MEHR', 'MXR0', 'JL00', 'MPVS', 'MPC0', 'EBB3', 'EFY0', 'EJB0', 'EQ00', 'ET00', 'EZL0', 'EZ00', 'MERC', 'MLGP', 'MPVC', 'MLXF', 'MT00', 'PX05', 'PB05', 'PV37', 'PX09', 'PY02', 'PY10', 'MJKG', 'AILB', 'AFPC', 'MML0', 'D000', 'AI00', 'AA00', 'ATFR', 'M000', 'J000', 'ATPR', 'MT0F', 'AFPI', 'AFCM', 'AFFN', 'ECK1', 'EEC0', 'AC00', 'PY24', 'MDD0', 'MEP0', 'PV03', 'MT0W', 'JP05', 'MPCG', 'PV07', 'PY30', 'MD00', 'AE00', 'DE00', 'EW00', 'EJ00', 'ENS0', 'EWL0', 'ERF0', 'EFN7', 'EFE1', 'EFE2', 'JQ05', 'EFP0', 'ED00', 'PP00', 'MEEW', 'ANNH', 'EMP6', 'ENG0', 'ETP0', 'EWJ0', 'JA03', 'JA34', 'JD01', 'MBPR', 'MC00', 'MCRA', 'MJKH', 'MPGB', 'MPPM', 'MPVW', 'MWDC', 'PS02', 'PB04', 'NW00', 'ACCH', 'ATWB', 'AIAH', 'AFWF', 'ATRJ', 'ADEA', 'EE00', 'ECM4', 'ERC1', 'AYWP', 'AIAR', 'ATWM', 'ATDB', 'SS20', 'AESC', 'EFD0', 'MFD0', 'EWU1', 'MP00', 'MSXH', 'MUSG', 'ESM0', 'EWC0', 'JV07', 'MDNS', 'EBB1', 'P000', 'MECM', 'MPGW', 'PY14', 'MMT0', 'MUTC', 'PX07', 'ERC5', 'AFFS', 'EMT0', 'EZS0', 'MBSD', 'AMUG', 'AKSN', 'AYPC', 'ADGG', 'ESI1', 'EMC2', 'FA00', 'MFDR', 'AMCR', 'PP02', 'PP01', 'DG02', 'C000', 'EAA1', 'ECM5', 'ALS0', 'AELH', 'PV09', 'JP00', 'AFWV', 'ERC3', 'PX00', 'JT00', 'PP03', 'MJLL', 'ERC8', 'JD00', 'JA02', 'MEHX', 'ATRE', 'ADWA', 'ENN1', 'A000', 'MAA0']
combined.CategoryCode = combined.CategoryCode.drop_duplicates(keep = 'first')

In [0]:
# Naive Bayes Classifier for Multinomial Models #

In [0]:
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

%time

y_pred = nb.predict(X_test)
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

In [0]:
# Linear Support Vector Machine #

In [0]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(X_train, y_train)

%time

y_pred = sgd.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

In [0]:
# Logistic Regression #

In [0]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])
logreg.fit(X_train, y_train)

%time

y_pred = logreg.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

In [0]:
# Doc2vec and Logistic Regression #

In [0]:
def label_sentences(corpus, label_type):
    labeled = []
    all_data = []
    for i, v in enumerate(corpus):
        label = label_type + '_' + str(i)
        labeled.append(doc2vec.TaggedDocument(v.split(), [label]))
    return labeled
X_train, X_test, y_train, y_test = train_test_split(combined.MRLineItemDescription, combined.CategoryCode, random_state=0, test_size=0.3)
X_train = label_sentences(X_train, 'Train')
X_test = label_sentences(X_test, 'Test')
all_data = X_train + X_test

In [0]:
# Doc2vec'i eğitirken, aşağıdaki parametreleri değiştirildi:

# dağıtılmış kelime torbası (DBOW) kullanıldı. -> dm=0 
# vektör boyutlu özellik vektörleri. -> vector_size=(300 , 300 )
# kaç tane "gürültü kelimesi" çizileceğini belirtir. -> negative=5 
# toplam frekansı bundan düşük olan tüm kelimeleri yok sayar. -> min_count=1
# başlangıç öğrenme oranı -> alpha=0.065 
# eğitim/devir sayısı -> 30

In [0]:
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, min_count=1, alpha=0.065, min_alpha=0.065)
model_dbow.build_vocab([x for x in tqdm(all_data)])

for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(all_data)]), total_examples=len(all_data), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

In [0]:
# Eğitimli doc2vec modelinden vektörler alınıp doc2vec özellikleri tarafından eğitilmiş 
# bir lojistik regresyon modeli elde edilir.

In [0]:
def get_vectors(model, corpus_size, vectors_size, vectors_type):
    vectors = np.zeros((corpus_size, vectors_size))
    for i in range(0, corpus_size):
        prefix = vectors_type + '_' + str(i)
        vectors[i] = model.docvecs[prefix]
    return vectors
    
train_vectors_dbow = get_vectors(model_dbow, len(X_train), 300, 'Train')
test_vectors_dbow = get_vectors(model_dbow, len(X_test), 300, 'Test')

In [0]:
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(train_vectors_dbow, y_train)
logreg = logreg.fit(train_vectors_dbow, y_train)
y_pred = logreg.predict(test_vectors_dbow)
my_tags = ['ATRX', 'DD00', 'AFGB', 'DB00', 'AMSC', 'AFCL', 'ATWP', 'AFWC', 'ATJS', 'AFTC', 'AYTC', 'AFCP', 'AY00', 'ANNM', 'AEFK', 'AFTS', 'MZW0', 'U000', 'ALLU', 'AFFT', 'ADDW', 'CE00', 'EWE1', 'ER00', 'EEA0', 'EF00', 'AYSS', 'EA00', 'PY22', 'EP00', 'PX01', 'MDHM', 'PF03', 'NND0', 'USME', 'AEFS', 'ADGS', 'ADDX', 'MLXR', 'ATFS', 'ADHD', 'AYVL', 'ANEJ', 'AFPP', 'AERA', 'AEDL', 'ADDZ', 'AFCA', 'MHLF', 'AYSF', 'ALST', 'MGED', 'ESL1', 'ETP3', 'ESL2', 'EMT2', 'ESL0', 'ES00', 'ECK0', 'EYW0', 'ENA0', 'EU00', 'MWBS', 'AYVR', 'MAHU', 'MPVE', 'MAHT', 'CY03', 'MPGF', 'PF02', 'MEC0', 'MPGC', 'PY26', 'MPGR', 'MAH0', 'ME00', 'SV00', 'JN01', 'MAHC', 'PY32', 'EVV1', 'JQ12', 'AYTA', 'AFFR', 'AFFL', 'AYML', 'ADRC', 'AYMP', 'AICM', 'AYMS', 'AESP', 'AYBD', 'AFFA', 'PY21', 'AIWS', 'AIWH', 'AFFW', 'EG00', 'ERG0', 'ELF1', 'EGL0', 'ELF2', 'EFC0', 'EWG1', 'ACE0', 'PX12', 'PB03', 'NNP0', 'MWKW', 'PV01', 'PV00', 'MPGL', 'JF05', 'ATPW', 'MFFW', 'ENF0', 'MAHV', 'ATIB', 'DD01', 'ATR0', 'ETV0', 'MPGH', 'MAHM', 'ATCT', 'AYPA', 'CY10', 'ADCA', 'DY00', 'ALLG', 'AICK', 'EFV5', 'EMT1', 'ALLS', 'PB00', 'PY05', 'MPGS', 'AAPG', 'AFMP', 'AFGG', 'MSXF', 'ADDS', 'EUY0', 'EMM0', 'MBA0', 'MEVH', 'ATRL', 'MDNE', 'NLLG', 'AFPS', 'EFW0', 'AFFC', 'ELF0', 'AEPC', 'ATWH', 'MERS', 'ATRT', 'AYPW', 'ATWC', 'MBS0', 'DY10', 'DG03', 'EB00', 'MEHU', 'DG00', 'AFMI', 'ADSM', 'ADDF', 'AMUM', 'MWDH', 'EH01', 'PY17', 'PY00', 'AAFS', 'EFV2', 'DY11', 'ATWF', 'AFPE', 'AFGP', 'AD00', 'AFFZ', 'CY17', 'MDND', 'PY01', 'DG01', 'SS01', 'CP00', 'NWC0', 'EL00', 'ERT0', 'ETP1', 'EWU0', 'FB02', 'MWR0', 'MEHR', 'MXR0', 'JL00', 'MPVS', 'MPC0', 'EBB3', 'EFY0', 'EJB0', 'EQ00', 'ET00', 'EZL0', 'EZ00', 'MERC', 'MLGP', 'MPVC', 'MLXF', 'MT00', 'PX05', 'PB05', 'PV37', 'PX09', 'PY02', 'PY10', 'MJKG', 'AILB', 'AFPC', 'MML0', 'D000', 'AI00', 'AA00', 'ATFR', 'M000', 'J000', 'ATPR', 'MT0F', 'AFPI', 'AFCM', 'AFFN', 'ECK1', 'EEC0', 'AC00', 'PY24', 'MDD0', 'MEP0', 'PV03', 'MT0W', 'JP05', 'MPCG', 'PV07', 'PY30', 'MD00', 'AE00', 'DE00', 'EW00', 'EJ00', 'ENS0', 'EWL0', 'ERF0', 'EFN7', 'EFE1', 'EFE2', 'JQ05', 'EFP0', 'ED00', 'PP00', 'MEEW', 'ANNH', 'EMP6', 'ENG0', 'ETP0', 'EWJ0', 'JA03', 'JA34', 'JD01', 'MBPR', 'MC00', 'MCRA', 'MJKH', 'MPGB', 'MPPM', 'MPVW', 'MWDC', 'PS02', 'PB04', 'NW00', 'ACCH', 'ATWB', 'AIAH', 'AFWF', 'ATRJ', 'ADEA', 'EE00', 'ECM4', 'ERC1', 'AYWP', 'AIAR', 'ATWM', 'ATDB', 'SS20', 'AESC', 'EFD0', 'MFD0', 'EWU1', 'MP00', 'MSXH', 'MUSG', 'ESM0', 'EWC0', 'JV07', 'MDNS', 'EBB1', 'P000', 'MECM', 'MPGW', 'PY14', 'MMT0', 'MUTC', 'PX07', 'ERC5', 'AFFS', 'EMT0', 'EZS0', 'MBSD', 'AMUG', 'AKSN', 'AYPC', 'ADGG', 'ESI1', 'EMC2', 'FA00', 'MFDR', 'AMCR', 'PP02', 'PP01', 'DG02', 'C000', 'EAA1', 'ECM5', 'ALS0', 'AELH', 'PV09', 'JP00', 'AFWV', 'ERC3', 'PX00', 'JT00', 'PP03', 'MJLL', 'ERC8', 'JD00', 'JA02', 'MEHX', 'ATRE', 'ADWA', 'ENN1', 'A000', 'MAA0']

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))